## Phase 1: Code for Data Loading, Cleaning & Feature Engineering
### Part 1: Setup and Initial Loading

In [1]:
# ---------------------------------------------------------------------------
# 1. SETUP & INITIAL DATA LOADING
# ---------------------------------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (14, 8) # Set default figure size

# Load the dataset from the CSV file into a pandas DataFrame
df = pd.read_csv('perth_property_data.csv')

# --- Initial Inspection ---

# Display a concise summary of the DataFrame.
print("\nDataFrame Info:")
df.info()


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42958 entries, 0 to 42957
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Listing_ID                 42958 non-null  int64  
 1   Price                      42958 non-null  int64  
 2   Agency_Name                42958 non-null  object 
 3   Postcode                   42958 non-null  int64  
 4   Address                    42958 non-null  object 
 5   Suburb                     42958 non-null  object 
 6   Longitude                  42958 non-null  float64
 7   Latitude                   42958 non-null  float64
 8   Property_Type              42958 non-null  object 
 9   Bedrooms                   42958 non-null  int64  
 10  Bathrooms                  42958 non-null  int64  
 11  Parking_Spaces             42958 non-null  int64  
 12  Date_Sold                  42958 non-null  object 
 13  Land_Size                  42

In [2]:
# Generate descriptive statistics for numerical columns.
print("\nDescriptive Statistics:")
print(df.describe())


Descriptive Statistics:
         Listing_ID         Price      Postcode     Longitude      Latitude  \
count  4.295800e+04  4.295800e+04  42958.000000  42958.000000  42958.000000   
mean   1.391644e+08  8.610860e+05   6083.335840    115.867264    -31.956828   
std    6.312142e+06  5.956430e+05     54.925532      0.077023      0.087968   
min    1.014242e+08  1.000000e+00   6003.000000    115.732150    -32.133780   
25%    1.375409e+08  5.100000e+05   6026.000000    115.804930    -32.034580   
50%    1.409794e+08  6.875000e+05   6064.000000    115.854050    -31.948990   
75%    1.432736e+08  9.800000e+05   6149.000000    115.923790    -31.886392   
max    1.457244e+08  6.350000e+06   6166.000000    116.064601    -31.774076   

           Bedrooms     Bathrooms  Parking_Spaces     Land_Size  \
count  42958.000000  42958.000000    42958.000000  4.295800e+04   
mean       3.366614      1.731831        2.039643  4.179946e+03   
std        0.878614      0.643763        1.129544  7.392181e+0

In [3]:
# --- Load and Apply Manual Corrections ---
df_raw = pd.read_csv('perth_property_data.csv')
try:
    # Load the corrections file
    df_corrections = pd.read_csv('corrections.csv')
    print(f"Found and loaded {len(df_corrections)} rules from corrections.csv.")
    
    df = df_raw.copy()
    
    # Step 1: Handle Deletions
    # Identify all Listing_IDs marked for deletion.
    ids_to_delete = df_corrections[df_corrections['New_Value'] == 'DELETE_ROW']['Listing_ID'].astype(int).tolist()
    
    if ids_to_delete:
        initial_rows = len(df)
        # Use .isin() to filter out the rows to be deleted. The ~ inverts the selection.
        df = df[~df['Listing_ID'].isin(ids_to_delete)]
        print(f"Deleted {initial_rows - len(df)} rows based on DELETE_ROW rules.")
    
    # Step 2: Handle Value Updates
    # Filter for rules that are not deletions.
    corrections_to_apply = df_corrections[df_corrections['New_Value'] != 'DELETE_ROW'].copy()
    # Ensure data types are correct for matching
    corrections_to_apply['Listing_ID'] = corrections_to_apply['Listing_ID'].astype(int)
    corrections_to_apply['New_Value'] = pd.to_numeric(corrections_to_apply['New_Value'])

    # Iterate through each correction rule and apply it using .loc for precision.
    for index, rule in corrections_to_apply.iterrows():
        listing_id = rule['Listing_ID']
        column = rule['Column_To_Correct']
        new_value = rule['New_Value']
        
        # This command finds the exact row(s) and column and sets the new value.
        df.loc[df['Listing_ID'] == listing_id, column] = new_value
        print(f"Updated Listing_ID {listing_id}: Set column '{column}' to {new_value}.")

except FileNotFoundError:
    print("Warning: corrections.csv not found. Proceeding with raw data.")
    df = df_raw.copy()

# --- Manual Correction Complete ---
# All subsequent cleaning and analysis will be performed on the 'df' DataFrame.
print(f"\nData ready for automated cleaning. Current rows: {len(df)}.")

Found and loaded 10 rules from corrections.csv.
Deleted 4 rows based on DELETE_ROW rules.
Updated Listing_ID 138928707: Set column 'Price' to 730000.
Updated Listing_ID 131703414: Set column 'Price' to 750000.
Updated Listing_ID 142122104: Set column 'Price' to 530000.
Updated Listing_ID 142973724: Set column 'Land_Size' to 404.
Updated Listing_ID 105535616: Set column 'Postcode' to 6152.
Updated Listing_ID 104755052: Set column 'Postcode' to 6014.

Data ready for automated cleaning. Current rows: 42954.


In [4]:
# ---------------------------------------------------------------------------
# 2. TECHNICAL CLEANING & FEATURE ENGINEERING
#
# Decision: Based on data validation, the extreme values for both 'Price' and 
# 'Land_Size' are considered legitimate and will be retained to ensure a
# complete market representation. Automated outlier removal will be skipped.
# ---------------------------------------------------------------------------

key_text_columns = ['Agency_Name', 'Primary_School_Name', 'Secondary_School_Name']

for col in key_text_columns:
    # Check if the column exists and is of object type before applying string operations.
    if col in df.columns and df[col].dtype == 'object':
        df.loc[:, col] = df[col].str.strip().str.lower()
        
# convert 'Date_Sold' column from an object (text) to a datetime object.

print("Starting technical cleaning...")
df['Date_Sold'] = pd.to_datetime(df['Date_Sold'], errors='coerce')

# Optional but recommended: Check for and handle any conversion errors.
if df['Date_Sold'].isnull().any():
    num_errors = df['Date_Sold'].isnull().sum()
    print(f"Warning: Found {num_errors} date(s) that could not be parsed. These rows will be dropped.")
    df.dropna(subset=['Date_Sold'], inplace=True)

print("Successfully converted 'Date_Sold' to datetime objects.")


# --- 2.2 Feature Engineering ---
# This step creates new, valuable features from existing data without altering it.
# We extract time-based features from the 'Date_Sold' column.

df['Sale_Year'] = df['Date_Sold'].dt.year
df['Sale_Month'] = df['Date_Sold'].dt.month
df['Sale_DayOfWeek'] = df['Date_Sold'].dt.dayofweek # Note: Monday=0, Sunday=6

print("Engineered new features: 'Sale_Year', 'Sale_Month', 'Sale_DayOfWeek'.")


# --- 2.3 Final Verification ---
# A final check to ensure our DataFrame is ready for analysis.

# We use .copy() here to create a definitive 'cleaned' version of the DataFrame.
# This can help prevent 'SettingWithCopyWarning' in later exploratory stages.
df_cleaned = df.copy()

print("\n--- Final DataFrame Info ---")
df_cleaned.info()

print("\n--- Final Descriptive Statistics ---")
print(df_cleaned.describe())

print("\nTechnical cleaning and feature engineering complete. The dataset is ready for EDA.")

Starting technical cleaning...
Successfully converted 'Date_Sold' to datetime objects.
Engineered new features: 'Sale_Year', 'Sale_Month', 'Sale_DayOfWeek'.

--- Final DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
Index: 42954 entries, 0 to 42954
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Listing_ID                 42954 non-null  int64         
 1   Price                      42954 non-null  int64         
 2   Agency_Name                42954 non-null  object        
 3   Postcode                   42954 non-null  int64         
 4   Address                    42954 non-null  object        
 5   Suburb                     42954 non-null  object        
 6   Longitude                  42954 non-null  float64       
 7   Latitude                   42954 non-null  float64       
 8   Property_Type              42954 non-null  object        
 9   Bedrooms   

/var/folders/sf/97mgdx1j24nbpzp1_h96gl_40000gn/T/ipykernel_21419/4092186627.py:19: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Date_Sold'] = pd.to_datetime(df['Date_Sold'], errors='coerce')


In [5]:
# Create a new categorical feature by combining bedrooms and bathrooms.
# This captures the "layout" or "floor plan" of the property as a single feature.
# We convert them to string to concatenate them.
df['Layout'] = df['Bedrooms'].astype(str) + 'b' + df['Bathrooms'].astype(str) + 'b'

print("Engineered new feature: 'Layout' (e.g., 3b2b, 4b2b).")

# Let's inspect the most common layouts
print("\nTop 15 most common property layouts:")
print(df['Layout'].value_counts().head(15))

Engineered new feature: 'Layout' (e.g., 3b2b, 4b2b).

Top 15 most common property layouts:
Layout
4b2b    12548
3b1b     9622
3b2b     8604
2b1b     3245
4b3b     1639
4b1b     1580
5b2b     1413
2b2b     1318
1b1b     1007
5b3b      907
3b3b      242
5b4b      171
6b3b      158
4b4b      127
6b2b      113
Name: count, dtype: int64


### Perth Property Data from CSV to MySQL

In [6]:
# --- Phase 1: Setup and Configuration ---
print("--- Phase 1: Setup and Configuration ---")

# --- Step 1: Import necessary libraries ---
import pandas as pd
from sqlalchemy import create_engine, text
import os

# --- Step 2: Database connection credentials ---
DB_USER = 'root'
DB_PASS = 'password' 
DB_HOST = 'localhost'
DB_PORT = '3306'
DB_NAME = 'perth_property_db' 

# --- Step 3: Create the SQLAlchemy engine ---
# This engine acts as the central point of contact between our script and the database.
try:
    db_connection_str = f'mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
    engine = create_engine(db_connection_str)
    
    # --- Step 4: Test the connection ---
    with engine.connect() as connection:
        print("Successfully connected to the MySQL database!")
        
except Exception as e:
    print(f"ERROR: Could not connect to the database. Please check your credentials and ensure MySQL is running. \n{e}")
    # We stop the script here if connection fails.
    raise

--- Phase 1: Setup and Configuration ---
Successfully connected to the MySQL database!


In [7]:
## use the df we cleaned before 
df_cleaned = df.copy()
# --- Prepare Dimension DataFrames ---
print("\n--- Preparing Dim DataFrames ---")

# --- Dimension 1: Layouts ---
# The combination of bedrooms and bathrooms is already unique by definition.
df_layouts = df_cleaned[['Bedrooms', 'Bathrooms', 'Layout']].drop_duplicates().reset_index(drop=True)
df_layouts = df_layouts.rename(columns={'Bedrooms': 'bedrooms', 'Bathrooms': 'bathrooms', 'Layout': 'layout_name'})
print(f"Created DIM_Layouts DataFrame with {len(df_layouts)} unique layouts.")

# Dimension 2: Suburbs
# CORRECTED: Drop duplicates based *only* on the 'Suburb' column to match the UNIQUE constraint in SQL.
df_suburbs = df_cleaned[['Suburb', 'Postcode']].drop_duplicates().reset_index(drop=True).rename(columns={'Suburb': 'suburb_name', 'Postcode': 'postcode'})
print(f"Prepared DIM_Suburbs with {len(df_suburbs)} unique suburbs.")
# Dimension 3: Agencies
df_agencies = pd.DataFrame(df_cleaned['Agency_Name'].dropna().unique(), columns=['agency_name'])
print(f"Prepared DIM_Agencies with {len(df_agencies)} unique agencies.")

# --- Dimension 4: Primary Schools ---
# CORRECTED: Explicitly drop duplicates based on the school name.
df_primary_schools = df_cleaned[['Primary_School_Name', 'Primary_School_ICSEA']].dropna(subset=['Primary_School_Name']).drop_duplicates(subset=['Primary_School_Name']).reset_index(drop=True)
df_primary_schools = df_primary_schools.rename(columns={'Primary_School_Name': 'primary_school_name', 'Primary_School_ICSEA': 'primary_school_icsea'})
print(f"Created DIM_Primary_Schools DataFrame with {len(df_primary_schools)} unique primary schools.")

# --- Dimension 5: Secondary Schools ---
# CORRECTED: Explicitly drop duplicates based on the school name.
df_secondary_schools = df_cleaned[['Secondary_School_Name', 'Secondary_School_ICSEA']].dropna(subset=['Secondary_School_Name']).drop_duplicates(subset=['Secondary_School_Name']).reset_index(drop=True)
df_secondary_schools = df_secondary_schools.rename(columns={'Secondary_School_Name': 'secondary_school_name', 'Secondary_School_ICSEA': 'secondary_school_icsea'})
print(f"Created DIM_Secondary_Schools DataFrame with {len(df_secondary_schools)} unique secondary schools.")



--- Preparing Dim DataFrames ---
Created DIM_Layouts DataFrame with 39 unique layouts.
Prepared DIM_Suburbs with 203 unique suburbs.
Prepared DIM_Agencies with 861 unique agencies.
Created DIM_Primary_Schools DataFrame with 344 unique primary schools.
Created DIM_Secondary_Schools DataFrame with 123 unique secondary schools.


In [8]:
# --- Phase 3: Load Data into MySQL ---
print("\n--- Loading Data into MySQL ---")

try:
    # Use a single connection for all operations in this block.
    with engine.connect() as connection:
        
        # --- Step 1: Recreate all tables from the SQL script ---
        # This is the corrected section.
        print("Executing create_tables.sql script...")
        sql_script_path = os.path.join('sql', 'create_tables.sql')
        
        with open(sql_script_path, 'r') as f:
            # Read the entire SQL script into a single string.
            sql_script = f.read()
            
            # Split the script into individual statements using the semicolon as a delimiter.
            sql_statements = sql_script.split(';')
            
            # Execute each statement one by one.
            # Begin a transaction to ensure all statements succeed or none do.
            with connection.begin():
                for statement in sql_statements:
                    # We must check if the statement is not just whitespace or comments.
                    if statement.strip():
                        connection.execute(text(statement))
                        
        print("Successfully executed create_tables.sql script (All tables dropped and recreated).")

        # --- Step 2: Load each dimension DataFrame into the corresponding table ---
        # This part remains the same as it correctly loads data table by table.
        with connection.begin(): # Use a new transaction for loading data
            df_layouts.to_sql('DIM_Layouts', con=connection, if_exists='append', index=False)
            print(f"Loaded {len(df_layouts)} records into DIM_Layouts.")

            df_suburbs.to_sql('DIM_Suburbs', con=connection, if_exists='append', index=False)
            print(f"Loaded {len(df_suburbs)} records into DIM_Suburbs.")
            
            df_agencies.to_sql('DIM_Agencies', con=connection, if_exists='append', index=False)
            print(f"Loaded {len(df_agencies)} records into DIM_Agencies.")

            df_primary_schools.to_sql('DIM_Primary_Schools', con=connection, if_exists='append', index=False)
            print(f"Loaded {len(df_primary_schools)} records into DIM_Primary_Schools.")

            df_secondary_schools.to_sql('DIM_Secondary_Schools', con=connection, if_exists='append', index=False)
            print(f"Loaded {len(df_secondary_schools)} records into DIM_Secondary_Schools.")

except Exception as e:
    print(f"ERROR during dimension table loading: \n{e}")
    # To help with debugging, we re-raise the exception after printing.
    raise


--- Loading Data into MySQL ---
Executing create_tables.sql script...
Successfully executed create_tables.sql script (All tables dropped and recreated).
Loaded 39 records into DIM_Layouts.
Loaded 203 records into DIM_Suburbs.
Loaded 861 records into DIM_Agencies.
Loaded 344 records into DIM_Primary_Schools.
Loaded 123 records into DIM_Secondary_Schools.


In [11]:
#
# Block 4: Prepare and Load the Fact Table
#
print("\n Loading the Fact Table ---")

try:
    # The outer 'with' statement manages the connection and a transaction.
    with engine.connect() as connection:
        
        # --- Step 1: Fetch dimension tables back from DB to get auto-generated IDs ---
        layouts_map = pd.read_sql("SELECT layout_id, layout_name FROM DIM_Layouts", connection)
        suburbs_map = pd.read_sql("SELECT suburb_id, suburb_name FROM DIM_Suburbs", connection)
        agencies_map = pd.read_sql("SELECT agency_id, agency_name FROM DIM_Agencies", connection)
        primary_schools_map = pd.read_sql("SELECT primary_school_id, primary_school_name FROM DIM_Primary_Schools", connection)
        secondary_schools_map = pd.read_sql("SELECT secondary_school_id, secondary_school_name FROM DIM_Secondary_Schools", connection)
        print("Fetched dimension tables back from DB to map foreign keys.")
        
        # --- Step 2: Merge foreign keys back into the main DataFrame ---
        df_merged = df_cleaned.copy()
        df_merged = pd.merge(df_merged, layouts_map, left_on='Layout', right_on='layout_name', how='left')
        df_merged = pd.merge(df_merged, suburbs_map, left_on='Suburb', right_on='suburb_name', how='left')
        df_merged = pd.merge(df_merged, agencies_map, left_on='Agency_Name', right_on='agency_name', how='left')
        df_merged = pd.merge(df_merged, primary_schools_map, left_on='Primary_School_Name', right_on='primary_school_name', how='left')
        df_merged = pd.merge(df_merged, secondary_schools_map, left_on='Secondary_School_Name', right_on='secondary_school_name', how='left')
        print("Foreign keys merged into the main DataFrame.")

        # --- Step 3: Debugging Checkpoint for NULL Foreign Keys ---
        print("\n--- DEBUG: Checking for NULLs in foreign key columns ---")
        fk_columns = ['suburb_id', 'agency_id', 'layout_id', 'primary_school_id', 'secondary_school_id']
        null_counts = df_merged[fk_columns].isnull().sum()
        print("Count of nulls in each foreign key column after merge:")
        print(null_counts)
        if null_counts.sum() == 0:
            print("SUCCESS: All foreign keys were merged successfully. No nulls found.")

        # --- Step 4: Prepare the final fact table DataFrame ---
        fact_table_columns = [
            'Listing_ID', 'Price', 'Address', 'Longitude', 'Latitude', 'Property_Type', 'Parking_Spaces', 
            'Date_Sold', 'Land_Size', 'Distance_to_CBD', 'Primary_School_Distance', 'Secondary_School_Distance',
            'suburb_id', 'agency_id', 'layout_id', 'primary_school_id', 'secondary_school_id'
        ]
        df_fact = df_merged[fact_table_columns].copy()
        
        df_fact.rename(columns={
            'Listing_ID': 'listing_id', 'Price': 'price', 'Address': 'address', 'Longitude': 'longitude',
            'Latitude': 'latitude', 'Property_Type': 'property_type', 'Parking_Spaces': 'parking_spaces',
            'Date_Sold': 'date_sold', 'Land_Size': 'land_size', 'Distance_to_CBD': 'distance_to_cbd',
            'Primary_School_Distance': 'primary_school_distance', 'Secondary_School_Distance': 'secondary_school_distance'
        }, inplace=True)
        
        initial_rows = len(df_fact)
        df_fact.dropna(subset=fk_columns, inplace=True)
        rows_dropped = initial_rows - len(df_fact)
        if rows_dropped > 0:
            print(f"Dropped {rows_dropped} rows due to null foreign keys before loading.")
        
        # --- Step 5: Load the fact table into the database ---
        # The 'to_sql' call is now directly inside the main 'with' block.
        # This block will automatically commit on success or rollback on error.
        df_fact.to_sql('FACT_Properties', con=connection, if_exists='append', index=False)
        # --- CRITICAL FIX: Explicitly commit the transaction after loading the fact table ---
        connection.commit()
        print(f"\nSuccessfully loaded {len(df_fact)} records into FACT_Properties!")
        
except Exception as e:
    print(f"\nERROR during fact table loading process: \n{e}")
    raise


 Loading the Fact Table ---
Fetched dimension tables back from DB to map foreign keys.
Foreign keys merged into the main DataFrame.

--- DEBUG: Checking for NULLs in foreign key columns ---
Count of nulls in each foreign key column after merge:
suburb_id              0
agency_id              0
layout_id              0
primary_school_id      0
secondary_school_id    0
dtype: int64
SUCCESS: All foreign keys were merged successfully. No nulls found.

Successfully loaded 42954 records into FACT_Properties!
